In [14]:
import pandas as pd
import numpy as np
from pandas import *
from scipy import stats

In [2]:
file = '../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Data/All Synchronized/P3_annos/annos_s20_2017-11-08/other_data/Combined_s20_2017-11-08.csv'

In [3]:
data = pd.read_csv(file)

/anaconda3/envs/caispp/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5,6,7,8,9,16,17,18,19,20,21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
data.head()

,Timestamp,participant,session,activity,difficulty,ros_ROBOT_STATE,ros_PARTICIPANT_STATE,transcript_spk_0,transcript_spk_1,transcript_spk_2,...,op_p2_hand_right_c17,op_p2_hand_right_x18,op_p2_hand_right_y18,op_p2_hand_right_c18,op_p2_hand_right_x19,op_p2_hand_right_y19,op_p2_hand_right_c19,op_p2_hand_right_x20,op_p2_hand_right_y20,op_p2_hand_right_c20
0,0.000000,3,s20_2017-11-08,1.0,2.0,game1_act1_inst1_8:Iwanttobringmoonrockshometo...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.033333,3,s20_2017-11-08,1.0,2.0,game1_act1_inst1_8:Iwanttobringmoonrockshometo...,NaN,Oh,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.066667,3,s20_2017-11-08,1.0,2.0,game1_act1_inst1_8:Iwanttobringmoonrockshometo...,NaN,Oh,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.100000,3,s20_2017-11-08,1.0,2.0,game1_act1_inst1_8:Iwanttobringmoonrockshometo...,NaN,Oh,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.133333,3,s20_2017-11-08,1.0,2.0,game1_act1_inst1_8:Iwanttobringmoonrockshometo...,NaN,my,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Precondition Calculations
- game start
    - new continuous instance of "game2_act1_inst0" in ros_ROBOT_STATE
- game end (moving on)
    - "Movingon" in ros_PARTICIPANT_STATE
- game end (completed successfully)
    - "GameCompleted" in ros_PARTICIPANT_STATE
- mistake made
    - new continuous instance of "Madeamistake in ros_PARTICIPANT_STATE"

In [5]:
new_columns = ['mistake_made', 'game_incorrect', 'game_correct', 'game_start']

for c in new_columns:
    data[c] = 0

In [6]:
for i,r in data.iterrows():
    ros_robot = r['ros_ROBOT_STATE']
    ros_participant = r['ros_PARTICIPANT_STATE']
    
    # new continuous instance in ros_ROBOT_STATE 
    if (isinstance(ros_robot, str)) and ((i==0) or (data.at[i-1, 'ros_ROBOT_STATE'] != ros_robot)): 
        # new game_start
        if ('game' in ros_robot) and ('act' in ros_robot) and  ('inst' in ros_robot):
            data.at[i, 'game_start'] = 1
        
    # new continuous instance in ros_PARTICIPANT_STATE
    if (isinstance(ros_participant, str)) and ((i==0) or (data.at[i-1, 'ros_PARTICIPANT_STATE'] != ros_participant)):
        # new game correct 
        if ('GameCompleted' in ros_participant):
            data.at[i, 'game_correct'] = 1
        
        # new game incorrect
        if ('Movingon' in ros_participant):
            data.at[i, 'game_incorrect'] = 1
        
        # mistake made
        if ('Madeamistake' in ros_participant):
            data.at[i, 'mistake_made'] = 1

In [36]:
fe_columns = ['time_since_robot', 'curr_comp_time', 'percentile_comp_time']

for c in fe_columns:
    data[c] = 0.0

last_robot = 0.0
last_game_start = 0.0
last_game_finished = 0.0
completion_times = []
game_started = False

for i,r in data.iterrows():
    if isinstance(r['ros_ROBOT_STATE'], str):
        last_robot = r['Timestamp']
    else:
        data.at[i, 'time_since_robot'] = r['Timestamp'] - last_robot

    if (r['game_start'] == 1):
        last_game_start = r['Timestamp']
        game_started = True
    
    if (game_started) and (last_game_finished <= last_game_start):
        data.at[i, 'curr_comp_time'] = r['Timestamp'] - last_game_start
        data.at[i, 'percentile_comp_time'] = stats.percentileofscore(completion_times, data.at[i, 'curr_comp_time'], kind = 'strict')
    
    if (r['game_correct'] == 1) or (r['game_incorrect'] == 1):
        last_game_finished = r['Timestamp']
        completion_times.append(data.at[i, 'curr_comp_time'])

In [37]:
# Sort Columns
cols = list(data.columns)

for c in fe_columns:
    cols.insert(0, cols.pop(cols.index(c)))

for c in new_columns:
    cols.insert(0, cols.pop(cols.index(c)))

cols.insert(0, cols.pop(cols.index('difficulty')))
cols.insert(0, cols.pop(cols.index('activity')))
cols.insert(0, cols.pop(cols.index('session')))
cols.insert(0, cols.pop(cols.index('participant')))
cols.insert(0, cols.pop(cols.index('Timestamp')))

data = data[cols]
data.to_csv('answer.csv', index=False)

100.0